# [Advent of Code 2019: Day 13](https://adventofcode.com/2019/day/13)

<h2>--- Day 13: Care Package ---</h2>
<p>As you ponder the solitude of space and the ever-increasing three-hour roundtrip for messages between you and Earth, you notice that the Space Mail Indicator Light is blinking. To help keep you sane, the Elves have sent you a care package.</p>
<p>It's a new game for the ship's <a href="https://en.wikipedia.org/wiki/Arcade_cabinet">arcade cabinet</a>! Unfortunately, the arcade is <em>all the way</em> on the other end of the ship. Surely, it won't be hard to build your own - the care package even
    comes with schematics.</p>
<p>The arcade cabinet runs <a href="https://adventofcode.com/2019/day/9">Intcode</a> software like the game the Elves sent (your puzzle input). It has a primitive screen capable of drawing square <em>tiles</em> on a grid. The software draws tiles to the
    screen with output instructions: every three output instructions specify the <code>x</code> position (distance from the left), <code>y</code> position (distance from the top), and <code>tile id</code>. The <code>tile id</code> is interpreted as follows:</p>
<ul>
    <li><code>0</code> is an <em>empty</em> tile. No game object appears in this tile.</li>
    <li><code>1</code> is a <em>wall</em> tile. Walls are indestructible barriers.</li>
    <li><code>2</code> is a <em>block</em> tile. Blocks can be broken by the ball.</li>
    <li><code>3</code> is a <em>horizontal paddle</em> tile. The paddle is indestructible.</li>
    <li><code>4</code> is a <em>ball</em> tile. The ball moves diagonally and bounces off objects.</li>
</ul>
<p>For example, a sequence of output values like <code>1,2,3,6,5,4</code> would draw a <em>horizontal paddle</em> tile (<code>1</code> tile from the left and <code>2</code> tiles from the top) and a <em>ball</em> tile (<code>6</code> tiles from the left
    and <code>5</code> tiles from the top).</p>
<p>Start the game. <em>How many block tiles are on the screen when the game exits?</em></p>

In [1]:
import os
from intcode import IntcodeComputer
    
class ArcadeGame():

    EMPTY = 0
    WALL = 1
    BLOCK = 2
    PADDLE = 3
    BALL = 4

    NEUTRAL = 0
    RIGHT = 1
    LEFT = -1
    
    def __init__(self, filename = ''):
        with open(filename) as file:
            game_data = file.read()
        self.iccp = IntcodeComputer(game_data)
        self.score = 0
        self.paddle_position = None
        self.tile_data = {}
            
    def play(self, quarters = None):
        if quarters != None:
            self.iccp.memory[0] = quarters            
        
        self.iccp.add_input(self.NEUTRAL)
        
        while True:
            self.iccp.run()
            x = self.iccp.output
            if x is None:
                break
                
            self.iccp.run()
            y = self.iccp.output

            if x == self.LEFT and y == self.NEUTRAL:
                self.iccp.run()
                self.score = self.iccp.output
            else:
                self.iccp.run()
                tile_id = self.iccp.output

                self.tile_data[y, x] = tile_id
                if tile_id == self.PADDLE:
                    self.paddle_position = x
                elif tile_id == self.BALL:
                    if self.paddle_position is not None:
                        if x > self.paddle_position:
                            self.iccp.add_input(self.RIGHT)
                        elif x < self.paddle_position:
                            self.iccp.add_input(self.LEFT)
                        else:
                            self.iccp.add_input(self.NEUTRAL)
        return 0

In [2]:
game = ArcadeGame("inputs/input_d13.txt")
game.play()
print(f'Number of block tiles: {len([v for v in game.tile_data.values() if v == game.BLOCK])}')

Number of block tiles: 420


<h2 id="part2">--- Part Two ---</h2>
<p>The game didn't run because you didn't put in any quarters. Unfortunately, you did not bring any <span title="You do have crew quarters, but they won't fit in the machine.">quarters</span>. Memory address <code>0</code> represents the number of quarters
    that have been inserted; set it to <code>2</code> to play for free.</p>
<p>The arcade cabinet has a <a href="https://en.wikipedia.org/wiki/Joystick">joystick</a> that can move left and right. The software reads the position of the joystick with input instructions:</p>
<ul>
    <li>If the joystick is in the <em>neutral position</em>, provide <code>0</code>.</li>
    <li>If the joystick is <em>tilted to the left</em>, provide <code>-1</code>.</li>
    <li>If the joystick is <em>tilted to the right</em>, provide <code>1</code>.</li>
</ul>
<p>The arcade cabinet also has a <a href="https://en.wikipedia.org/wiki/Display_device#Segment_displays">segment display</a> capable of showing a single number that represents the player's current score. When three output instructions specify <code>X=-1, Y=0</code>,
    the third output instruction is not a tile; the value instead specifies the new score to show in the segment display. For example, a sequence of output values like <code>-1,0,12345</code> would show <code>12345</code> as the player's current score.</p>
<p>Beat the game by breaking all the blocks. <em>What is your score after the last block is broken?</em></p>

In [3]:
game.play(2)
print(f'Score: {game.score}')

Score: 21651
